In [1]:
import pandas as pd
import numpy as np
import json
import random
import re

In [3]:
#Read data

df_position_sentiment = pd.read_csv(r'../data/qb/processed/combine_aspect_position_14000.csv')
#df_position_sentiment = pd.read_csv(r'./data/qb/processed/combine_aspect_position_0325_clean.csv')
#df_position_sentiment = pd.read_csv(r'./data/qb/processed/combine_aspect_position_0408_nolemma_clean.csv')

In [4]:
#Remove reviews without aspect sentiments
df_position_sentiment['Asp Sentiment'].replace('0', np.nan, inplace=True)
df_position_sentiment.dropna(subset=['Asp Sentiment'], inplace=True)

In [5]:
df_position_sentiment.head()

,Unnamed: 0,Unnamed: 0.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0.0,0.0,5464235433,Canada,20.01.5,1.0,2020-01-30,negative,neutral,latest app update. this update is terrible bas...,app,app,"7,9"
1,1.0,1.0,5464235433,Canada,20.01.5,1.0,2020-01-30,negative,negative,latest app update. this update is terrible bas...,updates,update,"11,16"
2,2.0,2.0,5466985498,USA,20.01.5,5.0,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
3,3.0,3.0,5460552711,USA,20.01.4,1.0,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,updates,update,"6,11"
4,4.0,4.0,5460552711,USA,20.01.4,1.0,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,experience,deleted,"23,29"


In [7]:
#df_position_sentiment[df_position_sentiment.AspectText == ]
#Drop records without the aspect text
print(len(df_position_sentiment))
df_position_sentiment.dropna(subset=['AspectText'], inplace=True)
print(len(df_position_sentiment))

14821
14821


In [8]:
#Check allowed sentiment categories
df_position_sentiment['Asp Sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [9]:
#recalculate position
for index, row in df_position_sentiment.iterrows():
    
    df_position_sentiment.loc[index,'Review'] = row['Review'].replace('/', '').replace('-', '')
    
    start_idx = row['Review'].find(row['AspectText'])
          
    df_position_sentiment.loc[index,'Positions'] = str(start_idx) + ',' + str(start_idx + len(row['AspectText']) - 1)
    

In [10]:
df_position_sentiment.to_csv(r'../data/qb/processed//combine_aspect_position_0408_clean_after.csv')

In [13]:
#Number of reviews with less than 200 words
(df_position_sentiment['Review'].str.len() <= 200).sum()

9716

In [14]:
df_position_sentiment.head()

,Unnamed: 0,Unnamed: 0.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0.0,0.0,5464235433,Canada,20.01.5,1.0,2020-01-30,negative,neutral,latest app update. this update is terrible bas...,app,app,"7,9"
1,1.0,1.0,5464235433,Canada,20.01.5,1.0,2020-01-30,negative,negative,latest app update. this update is terrible bas...,updates,update,"11,16"
2,2.0,2.0,5466985498,USA,20.01.5,5.0,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
3,3.0,3.0,5460552711,USA,20.01.4,1.0,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,updates,update,"6,11"
4,4.0,4.0,5460552711,USA,20.01.4,1.0,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,experience,deleted,"23,29"


In [15]:
#split data in - train test and dev
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    
    perm = np.random.permutation(np.arange(len(df)))
    m = len(df)
    
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end - 1] - 1]
    validate = df.iloc[perm[train_end:validate_end - 1] - 1]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [16]:
train, validate, test = train_validate_test_split(df_position_sentiment)

In [17]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
9417,9420.0,9447.0,gp:AOqpTOHZdHhdPN9L6WTYJakCXPaHrziUOVsH1Uuz26p...,English,19.8.0.2,1.0,2019-08-25,negative,negative,. crashes on launch after signing in with fing...,quality,crashes,"2,8"
8821,8823.0,8850.0,gp:AOqpTOGX3qKBFX485xqEGhX22q0TiClXaMrOka0mJrb...,English,19.11.0.7,3.0,2019-12-08,neutral,neutral,. i've been using qb online for almost three y...,quality,fix,"172,174"
11466,11506.0,11533.0,gp:AOqpTOF-xGc1oN4YEg2zGh1UxwMwM0I7nL5fEtiCxAb...,English,18.8.1,5.0,2018-10-08,positive,positive,. very excellent apps,app,apps,"17,20"
1238,1238.0,1255.0,3724610231,USA,18.12.1,1.0,2019-02-02,negative,negative,jan 2019 update is a fail. it no longer lets y...,updates,update,"9,14"
1128,1128.0,1144.0,3800117489,USA,18.12.2,1.0,2019-02-22,negative,negative,app crashes constantly. i used to love this ap...,quality,crashes,"4,10"


In [18]:
print(len(train))
print(len(test))
print(len(validate))

8891
2965
2963


### Generate files

In [19]:
def generate_input_files(d_type, data):

    #save reviews
    df_review = data[['Review']]
    df_review.to_csv(r'../data/qb/{}/review.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save aspects
    df_review = data[['AspectText']]
    df_review.to_csv(r'../data/qb/{}/term.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save positions
    df_review = data[['Positions']]
    df_review.to_csv(r'../data/qb/{}/position.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save label
    df_review = data[['Asp Sentiment']]
    df_review.to_csv(r'../data/qb/{}/label.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

In [20]:
generate_input_files('train', train)
generate_input_files('test', test)
generate_input_files('dev', validate)